# deTACTer: 버전별 시퀀스 데이터 통합 분석

이 노트북은 `deTACTer` 프로젝트에서 추출된 공격 시퀀스 데이터를 버전별로 비교하고 분석합니다.

### 주요 분석 내용
1. **팀별 시퀀스 생성 빈도**: 어떤 팀이 더 많은 공격 시퀀스를 만들어내는가?
2. **공격 성공률**: 어떤 팀의 시퀀스가 더 효과적인가? (슈팅/박스 진입 성공률)
3. **공격 템포 및 역동성**: 시퀀스의 진행 속도와 이동 거리는 어떠한가?
4. **전술적 시작 지점**: 공격이 주로 어디에서 시작되는가?
5. **액션 구성 분석**: 패스 중심인가, 드리블 중심인가?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font="Malgun Gothic")

In [ ]:
# 설정 및 버전 선택
with open('../config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

VERSION = config.get('version', 'v4.5')
print(f"현재 분석 버전: {VERSION}")

BASE_DIR = '..'
DATA_DIR = f"{BASE_DIR}/data/refined/{VERSION}/"
OUTPUT_DIR = f"{BASE_DIR}/results/anal/{VERSION}/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 1. 데이터 로드

In [ ]:
seq_df = pd.read_csv(DATA_DIR + 'attack_sequences.csv', encoding='utf-8-sig')
stats_df = pd.read_csv(DATA_DIR + 'attack_sequences_stats.csv', encoding='utf-8-sig', index_col=0)

# 팀 이름 매핑
team_map = seq_df.groupby('team_id')['team_name_ko'].first().to_dict()
stats_df['team_name'] = stats_df['team_id'].map(team_map).fillna(stats_df['team_id'])

display(stats_df.head())

## 2. 팀별 시퀀스 생성 수

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=stats_df, x='team_name', palette='viridis', order=stats_df['team_name'].value_counts().index)
plt.title(f'[{VERSION}] 팀별 공격 시퀀스 생성 수', fontsize=15)
plt.xticks(rotation=45)
plt.show()

## 3. 공격 성공률 분석

In [ ]:
stats_df['is_success'] = stats_df['outcome_result'].apply(lambda x: 1 if str(x).lower() in ['success', '1', '1.0'] else 0)
success_rate = stats_df.groupby('team_name')['is_success'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
success_rate.plot(kind='bar', color='salmon')
plt.title(f'[{VERSION}] 팀별 시퀀스 성공률', fontsize=15)
plt.ylabel('성공률')
plt.xticks(rotation=45)
plt.show()

## 4. 공격 템포 및 역동성

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.boxplot(data=stats_df, x='team_name', y='speed', ax=axes[0], palette='Set2')
axes[0].set_title('팀별 공격 템포 (속도)')
axes[0].tick_params(axis='x', rotation=45)

sns.boxplot(data=stats_df, x='team_name', y='distance', ax=axes[1], palette='Set3')
axes[1].set_title('팀별 시퀀스 총 이동 거리')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. 공격 시작 지점 분석

In [ ]:
plt.figure(figsize=(10, 7))
plt.gca().set_facecolor('#1a472a')
plt.plot([0, 1, 1, 0, 0], [0, 0, 1, 1, 0], 'w-', alpha=0.5)
plt.plot([0.5, 0.5], [0, 1], 'w--', alpha=0.3)

sns.kdeplot(data=stats_df, x='start_x', y='start_y', fill=True, cmap='YlGn', alpha=0.8, levels=20)
plt.title(f'[{VERSION}] 전체 공격 시퀀스 시작 지점 열지도', fontsize=15)
plt.xlim(-0.02, 1.02)
plt.ylim(-0.02, 1.02)
plt.show()

## 6. 액션 구성 프로필

In [ ]:
action_counts = seq_df.groupby(['team_name_ko', 'type_name']).size().unstack(fill_value=0)
action_pct = action_counts.div(action_counts.sum(axis=1), axis=0)

action_pct.plot(kind='bar', stacked=True, figsize=(13, 7), colormap='viridis')
plt.title(f'[{VERSION}] 팀별 시퀀스 내 액션 구성 비율', fontsize=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()